In [98]:
import csv
import os
import glob
import re
from pandas import DataFrame, Series
from openslide import open_slide
from PIL import Image
import timeit
import time
import math
import numpy as np
from scipy.ndimage.morphology import binary_fill_holes
from skimage.color import rgb2gray
from skimage.feature import canny
from skimage.morphology import binary_closing, binary_dilation, disk

In [2]:
train_paths = ["/scratch/ss4yd/chrc_data/train/Celiac/"]


images = {}
images_by_folder = {}
for train_path in train_paths:
    images_by_folder[str(train_path)] = []
    files = glob.glob(os.path.join(train_path, '*.svs'))
    for fl in files:
        flbase = os.path.basename(fl)
        flbase_noext = os.path.splitext(flbase)[0]
        images[flbase_noext]=fl
        images_by_folder[str(train_path)].append(flbase)

In [5]:
path_change_map = {}

for key in list(images_by_folder.keys()):
    temp = key.replace('chrc_data', 'chrc_data_patches')
    path_change_map[key] = temp

In [131]:
# images

In [52]:
overlap = 1500
patch_size = 3000

In [125]:
op_slide_img = open_slide(images['C03-10_02'])

In [126]:
step_size = patch_size - overlap

xlim = op_slide_img.level_dimensions[0][0]
ylim = op_slide_img.level_dimensions[0][1]
print("Dimensions x: " + str(xlim) + " y: " + str(ylim))

x_ind = math.ceil(xlim/(step_size))
y_ind = math.ceil(ylim/(step_size))

patches = []

img_y_left = ylim

y = 0 - step_size
for y_ in range(y_ind):
    x = 0-step_size
    y = y + step_size
    img_x_left = xlim
    img_y_left = img_y_left - step_size
    print('y - left: ' + str(img_y_left))
    if (img_y_left > 0 and img_y_left > patch_size):
        
        for x_ in range(x_ind):
            x = x + (step_size)
            img_x_left = img_x_left - step_size
            print(img_x_left)
            if (img_x_left > 0 and img_x_left > patch_size):
                patches.append(op_slide_img.read_region((x, y), 0, (patch_size, patch_size)))
            elif (img_x_left < patch_size and img_x_left > 0):
                x = xlim - patch_size
                patches.append(op_slide_img.read_region((x, y), 0, (patch_size, patch_size)))
                break
    
    elif (img_y_left > 0 and img_y_left < patch_size):
        y = ylim - patch_size
        for x_ in range(x_ind):
            x = x + (step_size)
            img_x_left = img_x_left - step_size
            print(img_x_left)
            if (img_x_left > 0 and img_x_left > patch_size):
                patches.append(op_slide_img.read_region((x, y), 0, (patch_size, patch_size)))
            elif (img_x_left < patch_size and img_x_left > 0):
                x = xlim - patch_size
                patches.append(op_slide_img.read_region((x, y), 0, (patch_size, patch_size)))
                break
        break
        
        
    


y - left: 5057
8084
6584
5084
3584
2084
y - left: 3557
8084
6584
5084
3584
2084
y - left: 2057
8084
6584
5084
3584
2084


In [127]:
for i in range(len(patches)):
    
    patches[i].save('test' + str(i)+'.jpg')
    

In [129]:
def create_patches(slide, patch_size, overlap):
    # steps to advance per axis with overlap
    step_size = patch_size - overlap
    
    # get dimensions of the image
    xlim = op_slide_img.level_dimensions[0][0]
    ylim = op_slide_img.level_dimensions[0][1]
    print("Dimensions x: " + str(xlim) + " y: " + str(ylim))
    
    # get the number of times to traverse each axis
    x_ind = math.ceil(xlim/(step_size))
    y_ind = math.ceil(ylim/(step_size))
    
    # initialize list to store patches
    patches = []
    
    # pixels left to traverse in the y-axis at the beginning of the traversal
    img_y_left = ylim
    # initialize the starting y corner
    y = 0 - step_size
    for y_ in range(y_ind):
        
        # initialize the starting x corner
        x = 0-step_size
        
        # advance the y axis (note: it starts with 0)
        y = y + step_size
        
        # pixels left to traverse in the x-axis at the beginning of the traversal
        img_x_left = xlim
        
        # update the number of pixels left to traverse
        img_y_left = img_y_left - step_size
        # print('y - left: ' + str(img_y_left))
        
        # if more than patch size left, get the (patch_size x patch_size) image
        if (img_y_left > 0 and img_y_left > patch_size):
            for x_ in range(x_ind):
                x = x + (step_size)
                img_x_left = img_x_left - step_size
                print(img_x_left)
                if (img_x_left > 0 and img_x_left > patch_size):
                    patches.append(op_slide_img.read_region(
                        (x, y), 0, (patch_size, patch_size)))
                elif (img_x_left < patch_size and img_x_left > 0):
                    x = xlim - patch_size
                    patches.append(op_slide_img.read_region((x, y), 0, (patch_size, patch_size)))
                    break
        # if less than patch size left, get the rest of the image, regardless of the overlap
        elif (img_y_left > 0 and img_y_left < patch_size):
            y = ylim - patch_size
            for x_ in range(x_ind):
                x = x + (step_size)
                img_x_left = img_x_left - step_size
                print(img_x_left)
                if (img_x_left > 0 and img_x_left > patch_size):
                    patches.append(op_slide_img.read_region(
                        (x, y), 0, (patch_size, patch_size)))
                elif (img_x_left < patch_size and img_x_left > 0):
                    x = xlim - patch_size
                    patches.append(op_slide_img.read_region(
                        (x, y), 0, (patch_size, patch_size)))
                    break
            break
    

In [85]:
def convert_to_3d_array(patch):
    rgb = patch.convert('RGB')
    return np.array(rgb)

tiles = [convert_to_3d_array(patch) for patch in patches]

In [130]:
create_patches(op_slide_img, patch_size=3000, overlap=1500)

Dimensions x: 9584 y: 6557
y - left: 5057
8084
6584
5084
3584
2084
y - left: 3557
8084
6584
5084
3584
2084
y - left: 2057
8084
6584
5084
3584
2084


In [87]:
def optical_density(tile):
    tile = tile.astype(np.float64)
    od = -np.log((tile+1)/240)
    return od

def keep_tile(tile, tile_size, tissue_threshold = 0.50):
    if tile.shape[0:2] == (tile_size, tile_size):
        print("inside if")
        tile_orig = tile
        tile = rgb2gray(tile)
        tile = 1 - tile
        
        tile = canny(tile)
        
        tile = binary_closing(tile, disk(10))
        tile = binary_dilation(tile, disk(10))
        tile = binary_fill_holes(tile)
        percentage1 = tile.mean()
        
        check1 = percentage1 >= tissue_threshold

        # Check 2
        # Convert to optical density values
        tile = optical_density(tile_orig)
        # Threshold at beta
        beta = 0.15
        tile = np.min(tile, axis=2) >= beta
        # Apply morphology for same reasons as above.
        tile = binary_closing(tile, disk(2))
        tile = binary_dilation(tile, disk(2))
        tile = binary_fill_holes(tile)
        percentage2 = tile.mean()
        check2 = percentage2 >= tissue_threshold
        print(percentage1, percentage2)
        return check1 and check2
    else:
        return False


In [97]:
keep_tile(tiles[0], 3000, tissue_threshold=0.3)

inside if


/apps/software/standard/core/anaconda3/4.4.0/lib/python3.6/site-packages/scipy/ndimage/measurements.py:431: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  safe = ((np.issubdtype(dt, int) and dt.itemsize <= int_size) or


0.07480377777777777 0.06350833333333333


False

In [88]:
tile_orig = tiles[1]
tile = optical_density(tile_orig)
tile

array([[[0.04691692, 0.04691692, 0.06010392],
        [0.04255961, 0.06010392, 0.06453852],
        [0.02531781, 0.04691692, 0.06010392],
        ...,
        [0.04255961, 0.05568891, 0.04255961],
        [0.04255961, 0.04255961, 0.03390155],
        [0.06453852, 0.06453852, 0.05568891]],

       [[0.05568891, 0.05568891, 0.06010392],
        [0.04691692, 0.06899287, 0.06899287],
        [0.04255961, 0.06010392, 0.06453852],
        ...,
        [0.04691692, 0.06010392, 0.04255961],
        [0.06453852, 0.06453852, 0.05568891],
        [0.06453852, 0.05129329, 0.05568891]],

       [[0.06453852, 0.06453852, 0.06453852],
        [0.06010392, 0.06899287, 0.06453852],
        [0.04255961, 0.06010392, 0.05568891],
        ...,
        [0.06010392, 0.06899287, 0.06453852],
        [0.06010392, 0.06899287, 0.06899287],
        [0.05129329, 0.06010392, 0.05568891]],

       ...,

       [[0.06899287, 0.05568891, 0.05568891],
        [0.04255961, 0.05129329, 0.04255961],
        [0.04255961, 0

In [89]:
# Threshold at beta
beta = 0.05
tile = np.min(tile, axis=2) >= beta
tile

array([[False, False, False, ..., False, False,  True],
       [ True, False, False, ..., False,  True,  True],
       [ True,  True, False, ...,  True,  True,  True],
       ...,
       [ True, False, False, ...,  True,  True, False],
       [ True,  True, False, ..., False,  True,  True],
       [ True,  True,  True, ..., False, False,  True]])

In [80]:
# Apply morphology for same reasons as above.
tile = binary_closing(tile, disk(2))
tile

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [81]:
tile = binary_dilation(tile, disk(2))
tile

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [82]:
tile = binary_fill_holes(tile)
tile

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])